In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Access and recess equation form the Parisian Alfonsines Tables
Hist_table=HTable.read(1372,format='dishas')
Hist_table

Superation,Tables [8]
deg,
Historical,void128
00 ; 27,"(1 ; , 00 ; 02,12,20) .. (8 ; , 00 ; 17,46,40)"
00 ; 28,"(1 ; , 00 ; 02,08,34) .. (8 ; , 00 ; 17,08,34)"
00 ; 29,"(1 ; , 00 ; 02,04,08) .. (8 ; , 00 ; 16,33,06)"
00 ; 30,"(1 ; , 00 ; 02,00,00) .. (8 ; , 00 ; 16,00,00)"
00 ; 31,"(1 ; , 00 ; 01,56,08) .. (8 ; , 00 ; 15,29,02)"
00 ; 32,"(1 ; , 00 ; 01,52,30) .. (8 ; , 00 ; 15,00,00)"
00 ; 33,"(1 ; , 00 ; 01,49,05) .. (8 ; , 00 ; 14,32,44)"
00 ; 34,"(1 ; , 00 ; 01,45,53) .. (8 ; , 00 ; 14,03,04)"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(8):
    for j in range(8):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')


In [4]:
for i in range(8):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        # initial value for superatio
        superatio=60*Hist_table[i][0]
                 
    for j in range(8):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
            # Initial values for elongation
            elong=Hist_table[0][1][j][0]
         
        # computation of the proportion
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            prop=elong/superatio
            Test_recomp[i][1][j][1]=prop*u.degree
            hist=Hist_table[i][1][j][1]
            Delta_recomp[i][1][j][1]=60*60*60*(hist-prop)

In [5]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(-4.625, 29.73739228648)

In [6]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,00 ; 27,00 ; 28,00 ; 29,00 ; 30,00 ; 31,00 ; 32,00 ; 33,00 ; 34
1 ;,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
7 ;,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
8 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-240.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Kammad short_TM.csv')
B.to_csv('Kammad short_TM.zip', index=False, compression=compression_opts)